In [ ]:
import re 
import requests
import pandas as pd
from bs4 import BeautifulSoup
from time import sleep
from selenium import webdriver
from PyDictionary import PyDictionary
import nltk
from nltk.tokenize import RegexpTokenizer
from nltk.corpus import stopwords
from nltk.corpus import wordnet
from gingerit.gingerit import GingerIt
from random import randint
from difflib import SequenceMatcher
from random import sample
import random

In [ ]:
def similar(a, b):
    ''' Function that takes a tow questions and returns similarity between them
    takes one argument :
    a - frist question
    b - second question'''
    return SequenceMatcher(None, a, b).ratio()

In [ ]:
def removeStopWords (qlist):
    ''' Function that takes a list of questions and returns unique words list without stop words
    takes one argument :
    qlist - list of questions'''
    
    # tokenizer splits tokens 
    tokenizer = RegexpTokenizer(r'\w+')
    
    # initiate empty list to fill it in the for loop  
    qlist2=[]
    
    # raverse for all questions
    for q in qlist:
        # append all words in the question in qlist2
        qlist2.append(tokenizer.tokenize(q))
    print(qlist2)

    # initiate empty list to fill it in the for loop 
    words=[]
    
    # Remove stop words from qlist2
    for ww in qlist2:
        for w in ww:
            if w not in stopwords.words('english'):
                if w.isupper() == False:
                    words.append(w)
                    
    # return all words in all questions without stop words and repetation  
    return unique(words)

In [ ]:
def unique(list1):
    ''' Function that takes list of words and returns unique words list
    takes one argument :
    list1 - list of words'''
    
    # intilize a null list
    unique_list = []
    # traverse for all elements
    for x in list1:
        # check if exists in unique_list or not
        if x not in unique_list:
            unique_list.append(x)
    for i in range(len(unique_list)):
        unique_list[i] = unique_list[i].lower()
    return unique_list

In [ ]:
def getSynynoms (unique_):
    ''' Function that takes unique words a list of lists that returns a list of all possible synynoms
    takes one argument :
    unique_ - list of orginal words'''
    
    # intilize three lists 
    syns = []
    allSyn = []
    notFound = []
    
    # traverse for all words in unique_
    for word in unique_:
        synset = wordnet.synsets(word)
        try:
            syns = []

            # range is the length of all possible synonyms for the word include the original word
            for i in range(len(synset[0].lemmas())):
                # we do not want to include the original word 
                if word != synset[0].lemmas()[i].name():
                    # add the synonym
                    syns.append(synset[0].lemmas()[i].name())
        except:
            allSyn.append(syns)
            notFound=[]
            continue
        # add the synonym in the list of all synynoms
        allSyn.append(syns)
    return allSyn

In [ ]:
def calssifySynynoms(unique_, allSyn):
    '''Function that takes a list of lists of synynoms
    and a list of words and then classify it as
    (noun, verb, adverb, adjective)
    takes two argument :
    unique_ - list of orginal words
    allSyn - a list of all possible synynoms'''
    
    # initiate empty list to fill it with noun synynoms, adj synynoms, verb synynoms and adver synynoms
    dictionary=PyDictionary()
    nounList = []
    adjList = []
    verbList = []
    adverbList = []

    for lis in allSyn:
        nouns = []
        adjs = []
        verbs = []
        adverbs = []
        for syn in lis:
            try:
                keys = dictionary.meaning(syn).keys()
            except:
                continue
            if 'Noun' in keys:
                nouns.append(syn)
            if 'Verb' in keys:
                verbs.append(syn)
            if 'Adjective' in keys:
                adjs.append(syn)
            if 'Adverb' in keys:
                adverbs.append(syn)

        if len(nouns) == 0:
            nouns.append('N/A')
        if len(adjs) == 0:
            adjs.append('N/A')
        if len(verbs) == 0:
            verbs.append('N/A')
        if len(adverbs) == 0:
            adverbs.append('N/A')

        nounList.append(nouns)
        adjList.append(adjs)
        verbList.append(verbs)
        adverbList.append(adverbs)

    nounSyn = pd.DataFrame()
    adjSyn = pd.DataFrame()
    verbSyn = pd.DataFrame()
    avSyn = pd.DataFrame()

    nounSyn['word'] = unique_
    adjSyn['word'] = unique_
    verbSyn['word'] = unique_
    avSyn['word'] = unique_
    ############################### Noun ####################################
    l1=[]
    for i in range(len(nounList)):
        try:
            l1.append(nounList[i][0])
        except:
            l1.append('N/A')  
    nounSyn['1st_synonym']= pd.Series(l1)

    l2=[]
    for i in range(len(nounList)):
        try:
            l2.append(nounList[i][1])
        except:
            l2.append('N/A')
    nounSyn['2nd_synonym']= pd.Series(l2)

    l3=[]
    for i in range(len(nounList)):
        try:
            l3.append(nounList[i][2])
        except:
            l3.append('N/A')
    nounSyn['3rd_synonym']= pd.Series(l3)


    ############################### Adjective ####################################
    l1=[]
    for i in range(len(adjList)):
        try:
            l1.append(adjList[i][0])
        except:
            l1.append('N/A')  
    adjSyn['1st_synonym']= pd.Series(l1)

    l2=[]
    for i in range(len(adjList)):
        try:
            l2.append(adjList[i][1])
        except:
            l2.append('N/A')
    adjSyn['2nd_synonym']= pd.Series(l2)

    l3=[]
    for i in range(len(adjList)):
        try:
            l3.append(adjList[i][2])
        except:
            l3.append('N/A')
    adjSyn['3rd_synonym']= pd.Series(l3)


    ############################### Verb ####################################
    l1=[]
    for i in range(len(verbList)):
        try:
            l1.append(verbList[i][0])
        except:
            l1.append('N/A')  
    verbSyn['1st_synonym']= pd.Series(l1)

    l2=[]
    for i in range(len(verbList)):
        try:
            l2.append(verbList[i][1])
        except:
            l2.append('N/A')
    verbSyn['2nd_synonym']= pd.Series(l2)

    l3=[]
    for i in range(len(verbList)):
        try:
            l3.append(verbList[i][2])
        except:
            l3.append('N/A')
    verbSyn['3rd_synonym']= pd.Series(l3)


    ############################### Adverb ####################################
    l1=[]
    for i in range(len(adverbList)):
        try:
            l1.append(adverbList[i][0])
        except:
            l1.append('N/A')  
    avSyn['1st_synonym']= pd.Series(l1)

    l2=[]
    for i in range(len(adverbList)):
        try:
            l2.append(adverbList[i][1])
        except:
            l2.append('N/A')
    avSyn['2nd_synonym']= pd.Series(l2)

    l3=[]
    for i in range(len(adverbList)):
        try:
            l3.append(adverbList[i][2])
        except:
            l3.append('N/A')
    avSyn['3rd_synonym']= pd.Series(l3)

   
    return (nounSyn,adjSyn,verbSyn,avSyn)

In [ ]:
def cleanDictionay (dictionary):
    '''Function to clean the synynom dictionary 
    takes one argument :
    dictionary - synynom dictionary'''
    
    # removing irrelevant words 
    dirty = ['one','two','three','four','five','six','seven','eight','nine','carbon','milliliters','liters','potassium',
    'second','gastrointestinal','litre','oxygen','min','became','eye','full','elevated','radius','law','years','fall',
    'bone','less','year']
    dictionary['word'] = dictionary['word'].apply(lambda x : x if x not in dirty else '2')

    # remove words of length of 2 ( of, in, to)
    dictionary['word'] = dictionary['word'].apply(lambda x : x if len(x)>2 else '2')# when we replace it by a number the next line 
                                                                                    # will delete it automatically since it is a number 
    # remove digits from the synynom dictionary
    dictionary = dictionary[~dictionary.word.str.contains(r'[0-9]')]
    
    # remove words that doesn't have synynoms
    dictionary = dictionary[dictionary['1st_synonym']!='N/A']
    
    # correct the syntax of the word
    dictionary['1st_synonym'] = dictionary['1st_synonym'].map(lambda x:str(x).replace('_',' '))
    dictionary['2nd_synonym'] = dictionary['2nd_synonym'].map(lambda x:str(x).replace('_',' '))
    dictionary['3rd_synonym'] = dictionary['3rd_synonym'].map(lambda x:str(x).replace('_',' '))
    
    
    
    return dictionary

In [ ]:
def suggestSynynoms(qlist,nounSyn,adjSyn,verbSyn,avSyn):
    ''' Function that takes a list of questions and returns the question with a new replaced synynom
    takes one argument :
    qlist - list of questions'''

    # Dectionaries that we created it before from calssifySynynoms function
#     nounSyn=pd.read_excel('nounsDictionary3.xlsx')
#     adjSyn=pd.read_excel('adjectiveDictionary3.xlsx')
#     verbSyn=pd.read_excel('verbDictionary3.xlsx')
#     avSyn=pd.read_excel('adverbDictionary3.xlsx')
    lstFound = []
    tokenizer = RegexpTokenizer(r'\w+|[^\w\s]+')
    
    oun = list(nounSyn['word'].unique())
    oun.extend(list(adjSyn['word'].unique()))
    oun.extend(list(verbSyn['word'].unique()))
    oun.extend(list(avSyn['word'].unique()))
    o = []
    # traverse for all elements
    for x in oun:
        # check if exists in unique_list or not
        if x not in o:
            o.append(x)
            
    for q in qlist:
        q1 = tokenizer.tokenize(q.lower())
        # traverse for all words in q1
        for i in q1:
                if i in o:
                    lstFound.append(i)
    
    numToReplace = randint(1, len(lstFound))
    lstToReplace = random.sample(lstFound, numToReplace)
    tokenizer = RegexpTokenizer(r'\w+|[^\w\s]+')
    newq=''
    new_qlist=[]
    q_with_capital=''
    for q in qlist:
#         q2 = tokenizer.tokenize(q)
#         for word in q2:
#             # if it is true it will not convert it to lowercase
#             if (word.isupper() == True)&(word != 'A'):
#                 q_with_capital=q_with_capital+word
#                 q_with_capital=q_with_capital+' '
#             else:
#                 q_with_capital=q_with_capital+word.lower()
#                 q_with_capital=q_with_capital+' '

        q1 = tokenizer.tokenize(q.lower())
        for i in q1:
            if i not in o or i not in lstToReplace:
                newq=newq+i
                newq=newq+' '
            
            elif i in o and i in lstToReplace:
                text = nltk.word_tokenize(q)
                result = nltk.pos_tag(q1)
                result = [e for e in result if e[0].lower() == i]
                type_= result[0][1]
                
                if type_ in ['NN','NNS','NNP','NN','NNPS']:
                    try:
                        #####
                        all_syn= list(nounSyn.loc[nounSyn['word']==i].iloc[0])
                        liss= [r for r in all_syn if r !='N/A' and r!=i]
                        sample(liss,1)
                        #####
                        newq=newq+str(sample(liss,1)[0])
                        newq=newq+' '
                    except:
                        newq=newq+i
                        newq=newq+' '


                elif type_ in ['JJ','JJR','JJS']:
                        try:
                             #####
                            all_syn= list(adjSyn.loc[adjSyn['word']==i].iloc[0])
                            liss= [r for r in all_syn if r !='N/A' and r!=i]
                            sample(liss,1)
                            #####
                            newq=newq+str(sample(liss,1)[0])
                            newq=newq+' '
                        except:
                            newq=newq+i
                            newq=newq+' '

                elif type_ in ['VB','VBD','VBG','VBN','VBP','VBZ']:
                    try:
                        #####
                        all_syn= list(verbSyn.loc[verbSyn['word']==i].iloc[0])
                        liss= [r for r in all_syn if r !='N/A' and r!=i]
                        sample(liss,1)
                        #####
                        newq=newq+str(sample(liss,1)[0])
                        newq=newq+' '
                    except:
                        newq=newq+i
                        newq=newq+' '
                    

                elif type_ in ['RB','RBR','RBS']:
                    try:
                        #####
                        all_syn= list(avSyn.loc[avSyn['word']==i].iloc[0])
                        liss= [r for r in all_syn if r !='N/A' and r!=i]
                        sample(liss,1)
                        #####
                        newq=newq+str(sample(liss,1)[0])
                        newq=newq+' '
                    except:
                        newq=newq+i
                        newq=newq+' '
                    
                else:
                    newq=newq+i
                    newq=newq+' '

        new_qlist.append(newq)
        newq=''
    return new_qlist

In [ ]:
def suggestVariable (q):
    ''' Function that takes a question and returns the question with a new replaced variable
    takes one argument :
    q - question'''
    
    sentences = list(pd.read_excel('sent_var.xlsx')['sentences'])
    variables = pd.read_excel('sent_var.xlsx').drop('Unnamed: 0', axis = 1)
    s =''
    
    for i in range(0,len(sentences)):
        if sentences[i] in q:
            replacement = variables[variables['sentences'] == sentences[i]]['variables'].iloc[0]
            q = q.replace(sentences[i], replacement)
            break
    
    # lists for ages 
    age1 = [1,2,3,4,5]
    age2 = [6,7,8,9,10,11,12,13]
    age3 = [14,15,16,17,18,19]
    age4 = [x for x in range (20,30)]
    age5 = [x for x in range (30,40)]
    age6 = [x for x in range (40,50)]
    age7 = [x for x in range (50,65)]
    age8 = [x for x in range (65,120)]
    newAge = 0
    
    # replacing ages 
    if 'year-old' in q:
        try:
            age = int(re.findall(r'\d+', q.split('year-old')[0])[0])
        except:
            return q
            
        if age in age1:
            newAge = sample(age1,1)
        elif age in age2:
            newAge = sample(age2,1)
        elif age in age3:
            newAge = sample(age3,1)
        elif age in age4:
            newAge = sample(age4,1)
        elif age in age5:
            newAge = sample(age5,1)
        elif age in age6:
            newAge = sample(age6,1)
        elif age in age7:
            newAge = sample(age7,1)
        elif age in age8:
            newAge = sample(age8,1)
        q = q.replace(str(age),str(newAge[0]))
    return q

In [ ]:
# gender function will specify the gender mentioned in a specific question

def gender(question):
    """ Specify the gender of a specific question
    Take one argument 
    question- one question 
    """
    all_genders= ['she','her','hers','he','him','his']
    # list of female pronouns
    female_pronouns= ['she','her','hers']
    # list of male pronouns
    male_pronouns= ['he','him','his']
    # list of possible synonymous of male 
    male = ['male','boy','youngman','man']
    # list of possible synonymous of female
    female = ['female','girl','lady','woman']
    # list of possible words of unknown gender
    unknown= ['patient','someone']
    # list of possible words of plural
    plural= ['patients','men','women']
    

    # tokenizer splits tokens 
    tokenizer = RegexpTokenizer(r'\w+')
    q = tokenizer.tokenize(question)
    
    # traverse for all words in q
    for i in q:
        
        # traverse for all words in male list
        if i in male:
            for w in q:
                if w in female:
                    # in case there is male and female in the question
                    return ('plural')
            return('male')
        
        # traverse for all words in female list
        if i in female:
            for u in q:
                if u in male:
                    # in case there is male and female in the question
                    return ('plural')      
            return('female')
        
        if i in unknown:
            # if the first word is in unknown list it will checks the next word if it is in female_pronouns list or male_pronouns list 
            for e in q:
                if e in all_genders:
                    if e in female_pronouns:
                        return('female')
                    elif e in male_pronouns:
                        return ('male')
        if i in plural:
            return('plural')
                
    return(False)

In [ ]:
# grammar checker function will do the folowing : 
# 1. Editing gender pronouns
# 2. Editing identifiers
# 3. Editing letter case
# 4. Control spacing
# 5. Correcting spelling and grammar
  
def grammar_checker(q):
    """ Editing gender pronouns, identifiers and letter case. Control spacing and correcting spelling and grammar mistakes
   
    Take one argument 
    q- list of questions
    """
    # list of subject pronouns
    l1= ['I','you','he', 'she', 'it','we','they']
    # list of object pronouns
    l2= ['me','you','him','her','it','us','them']
    # list of possessive adjectives 
    l3= ['my','your','his','her','its','our','their']
    # list of possessive pronouns
    l4= ['mine','yours','his','hers','its','ours','theirs']
    
    # assign GingerIt() to parser
    parser = GingerIt()
    
    # create empty list 
    list_of_questions=[]
    
    # for loop to enter each question in the list
    for index in range(len(q)):
        
        # change the type of question to string
        question=str(q[index])
        
        # call gender function to know the gender mentioned in the question
        gender_=gender(question)
        
        # splits tokens by white space 
        tokenizer = RegexpTokenizer(r'\w+|[^\w\s]+')
        q1= tokenizer.tokenize(question)
        
        # if gender = female, change the pronouns to: she, her, hers 
        if gender_ == 'female':
            for i in q1:
                if i in l1:
                    question=question.replace(i,'she')
                elif i in l2:
                    question=question.replace(i,'her')
                elif i in l3:
                    question=question.replace(i,'her')
                elif i in l4:
                    question=question.replace(i,'hers')
                    
        # if gender = male, change the pronouns to: he, him, his
        elif gender_ == 'male': 
            for i in q1:
                if i in l1:
                    question=question.replace(i,'he')
                elif i in l2:
                    question=question.replace(i,'him')
                elif i in l3:
                    question=question.replace(i,'his')
                elif i in l4:
                    question=question.replace(i,'his')
                    
         # if it is plural, change the pronouns to: they, them, their, theirs
        elif gender_ == 'plural': 
            for i in q1:
                if i in l1:
                    question=question.replace(i,'they')
                elif i in l2:
                    question=question.replace(i,'them')
                elif i in l3:
                    question=question.replace(i,'their')
                elif i in l4:
                    question=question.replace(i,'theirs')
        try: 
            # change first character after dot to uppercase and apply the GingerIt() on the question               
            capitalize=". ".join(i.capitalize() for i in parser.parse(question)['result'].split(". "))
            # add question to list_of_questions list    
            list_of_questions.append(capitalize)
        except:
            # add question to list_of_questions list 
            try:
                list_of_questions.append(parser.parse(question)['result'])
            except:
                list_of_questions.append(question)
    # return the new list 
    return (list_of_questions)